![iceberg-logo](https://www.apache.org/logos/res/iceberg/iceberg.png)

### [Docker, Spark, and Iceberg: The Fastest Way to Try Iceberg!](https://tabular.io/blog/docker-spark-and-iceberg/)

In [1]:
!pip install pyspark-opendic==0.1.4

In [1]:
from pyspark.sql import SparkSession

def read_secret(secret_name):
    """ Get `secret_name` from docker-compose secret store"""
    secret_path = f"/run/secrets/{secret_name}"
    try:
        with open(secret_path, "r") as f:
            return f.read().strip()  # Remove any trailing newline
    except FileNotFoundError:
        print(f"Secret {secret_name} not found.")
        return None


## DEFINE SENSITIVE VARIABLES
POLARIS_CATALOG_NAME = 'polaris'
ENGINEER_CLIENT_ID = read_secret("engineer_client_id")
ENGINEER_CLIENT_SECRET = read_secret("engineer_client_secret")
HR_CLIENT_ID = 'cf4970adffcea82f'
HR_CLIENT_SECRET= '6d5c2b8c9f4f9bd2a5c6112da6cb0359'

def create_session(client_id, client_secret, scope):
    spark = (SparkSession.builder
        .config("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.7.0,software.amazon.awssdk:bundle:2.28.17,software.amazon.awssdk:url-connection-client:2.28.17")
        .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
        .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog")
        .config("spark.sql.catalog.polaris", "org.apache.iceberg.spark.SparkCatalog")
        .config("spark.sql.catalog.polaris.type", "rest")
        .config("spark.sql.catalog.polaris.warehouse", POLARIS_CATALOG_NAME)
        .config("spark.sql.catalog.polaris.uri", 'http://polaris:8181/api/catalog')
        .config("spark.sql.catalog.polaris.credential", f"{client_id}:{client_secret}")
        .config("spark.sql.catalog.polaris.scope", 'PRINCIPAL_ROLE:ALL')
        .config("spark.sql.defaultCatalog", "polaris")
        .config("spark.sql.catalogImplementation", "in-memory")
        .config("spark.driver.extraJavaOptions", "-Divy.cache.dir=/tmp -Divy.home=/tmp")
        .config("spark.sql.catalog.polaris.token-refresh-enabled", "true")
        .config("spark.sql.catalog.polaris.header.X-Iceberg-Access-Delegation", 'vended-credentials')
        .config("spark.sql.catalog.polaris.io-impl", "org.apache.iceberg.io.ResolvingFileIO")
        .config("spark.history.fs.logDirectory", "/home/iceberg/spark-events")).getOrCreate()
        
    print("Spark Running")
    return spark


## Start Spark Session
spark = create_session(ENGINEER_CLIENT_ID, ENGINEER_CLIENT_SECRET, 'PRINCIPAL_ROLE:ALL')
spark

Spark Running


25/03/26 20:15:58 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


## Setup polaris catalog

In [3]:
%%sql
CREATE NAMESPACE IF NOT EXISTS ANDREAS_DEV

25/03/26 10:08:31 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
25/03/26 10:08:31 WARN RESTSessionCatalog: Iceberg REST client is missing the OAuth2 server URI configuration and defaults to http://polaris:8181/api/catalog/v1/oauth/tokens. This automatic fallback will be removed in a future Iceberg release.It is recommended to configure the OAuth2 endpoint using the 'oauth2-server-uri' property to be prepared. This warning will disappear if the OAuth2 endpoint is explicitly configured. See https://github.com/apache/iceberg/issues/10537


++
||
++
++

In [2]:
%%sql
CREATE NAMESPACE IF NOT EXISTS ANDREAS_DEV.PUBLIC

25/03/26 17:10:03 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
25/03/26 17:10:04 WARN RESTSessionCatalog: Iceberg REST client is missing the OAuth2 server URI configuration and defaults to http://polaris:8181/api/catalog/v1/oauth/tokens. This automatic fallback will be removed in a future Iceberg release.It is recommended to configure the OAuth2 endpoint using the 'oauth2-server-uri' property to be prepared. This warning will disappear if the OAuth2 endpoint is explicitly configured. See https://github.com/apache/iceberg/issues/10537


++
||
++
++

In [3]:
%%sql
SHOW NAMESPACES IN ANDREAS_DEV

namespace
ANDREAS_DEV.PUBLIC


## Setting up pyspark-opendic

In [4]:
from pyspark_opendic.catalog import OpenDicCatalog

# Init polarisx catalog
POLARIS_URI= "http://polaris:8181/api"
catalog = OpenDicCatalog(spark, POLARIS_URI)
print("Catalog initialized")

Catalog initialized


In [5]:
catalog.sql("USE polaris")
catalog.sql("CREATE NAMESPACE IF NOT EXISTS ANDREAS_DEV")
catalog.sql("CREATE NAMESPACE IF NOT EXISTS ANDREAS_DEV.PUBLIC")
catalog.sql("SHOW NAMESPACES").show()

25/03/13 15:21:45 WARN RESTSessionCatalog: Iceberg REST client is missing the OAuth2 server URI configuration and defaults to http://polaris:8181/api/catalog/v1/oauth/tokens. This automatic fallback will be removed in a future Iceberg release.It is recommended to configure the OAuth2 endpoint using the 'oauth2-server-uri' property to be prepared. This warning will disappear if the OAuth2 endpoint is explicitly configured. See https://github.com/apache/iceberg/issues/10537


+-----------+
|  namespace|
+-----------+
|ANDREAS_DEV|
+-----------+



In [7]:
catalog.sql(
 """CREATE OPEN function my_function 
    props {
        "args": {
            "arg1": "string",
            "arg2": "number"
        },
        "language": "sql"}"""
)

ValidationError: 1 validation error for CreateUdoRequest
udo
  Field required [type=missing, input_value={'object': Udo(type='func...ne, entityVersion=None)}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing

## Create a schema, an object and a mapping example

In [4]:
%%sql

CREATE DATABASE IF NOT EXISTS SYSTEM

++
||
++
++

In [5]:
%%sql 

use database SYSTEM;

++
||
++
++

In [3]:
%%sql

show tables

namespace,tableName,isTemporary
SYSTEM,python_function,False
SYSTEM,python_function2,False
SYSTEM,function,False
SYSTEM,andreas_function,False


In [4]:
%%sql

DESCRIBE EXTENDED andreas_function

col_name,data_type,comment
args,"map<string,struct<tag:int,field0:string,field1:int>>",None
runtime,string,None
language,string,None
comment,string,None
def,string,None
client_version,string,None
,,
# Metadata Columns,,
_spec_id,int,
_partition,struct<>,


In [23]:
%%sql
select * from SYSTEM.python_function

id,name,return_type,args,def,handler,language,runtime,packages
1,calculate_age,INT,birth_date DATE,"from datetime import date def calculate_age(birth_date): today = date.today() age = today.year - birth_date.year - ((today.month, today.day) < (birth_date.month, birth_date.day)) return age",calculate_age,python,3.10,datetime
1,calculate_age,INT,birth_date DATE,"def calculate_age(birth_date): from datetime import date today = date.today() age = today.year - birth_date.year - ((today.month, today.day) < (birth_date.month, birth_date.day)) return age",calculate_age,python,3.10,datetime


## Load One Month of NYC Taxi/Limousine Trip Data

For this notebook, we will use the New York City Taxi and Limousine Commision Trip Record Data that's available on the AWS Open Data Registry. This contains data of trips taken by taxis and for-hire vehicles in New York City. We'll save this into an iceberg table called `taxis`.

To be able to rerun the notebook several times, let's drop the table if it exists to start fresh.

In [6]:
%%sql

CREATE DATABASE IF NOT EXISTS nyc

++
||
++
++

In [12]:
%%sql

DROP TABLE IF EXISTS nyc.taxis

++
||
++
++

In [13]:
df = spark.read.parquet("/home/iceberg/data/yellow_tripdata_2021-04.parquet")
df.write.saveAsTable("nyc.taxis")

In [9]:
%%sql

DESCRIBE EXTENDED nyc.taxis

col_name,data_type,comment
VendorID,bigint,None
tpep_pickup_datetime,timestamp_ntz,None
tpep_dropoff_datetime,timestamp_ntz,None
passenger_count,double,None
trip_distance,double,None
RatecodeID,double,None
store_and_fwd_flag,string,None
PULocationID,bigint,None
DOLocationID,bigint,None
payment_type,bigint,None


In [10]:
%%sql

SELECT COUNT(*) as cnt
FROM nyc.taxis

cnt
2171187


## Schema Evolution

Adding, dropping, renaming, or altering columns is easy and safe in Iceberg. In this example, we'll rename `fare_amount` to `fare` and `trip_distance` to `distance`. We'll also add a float column `fare_per_distance_unit` immediately after `distance`.

In [8]:
%%sql

ALTER TABLE nyc.taxis RENAME COLUMN fare_amount TO fare

++
||
++
++

In [9]:
%%sql

ALTER TABLE nyc.taxis RENAME COLUMN trip_distance TO distance

++
||
++
++

In [10]:
%%sql

ALTER TABLE nyc.taxis ALTER COLUMN distance COMMENT 'The elapsed trip distance in miles reported by the taximeter.'

++
||
++
++

In [11]:
%%sql

ALTER TABLE nyc.taxis ALTER COLUMN distance TYPE double;

++
||
++
++

In [12]:
%%sql

ALTER TABLE nyc.taxis ALTER COLUMN distance AFTER fare;

++
||
++
++

In [13]:
%%sql

ALTER TABLE nyc.taxis
ADD COLUMN fare_per_distance_unit float AFTER distance

++
||
++
++

Let's update the new `fare_per_distance_unit` to equal `fare` divided by `distance`.

In [14]:
%%sql

UPDATE nyc.taxis
SET fare_per_distance_unit = fare/distance

++
||
++
++

In [15]:
%%sql

SELECT
VendorID
,tpep_pickup_datetime
,tpep_dropoff_datetime
,fare
,distance
,fare_per_distance_unit
FROM nyc.taxis

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,fare,distance,fare_per_distance_unit
1,2021-04-01 00:00:18,2021-04-01 00:21:54,25.5,8.4,3.0357143878936768
1,2021-04-01 00:42:37,2021-04-01 00:46:23,5.0,0.9,5.55555534362793
1,2021-04-01 00:57:56,2021-04-01 01:08:22,11.5,3.4,3.382352828979492
1,2021-04-01 00:01:58,2021-04-01 00:54:27,44.2,0.0,None
2,2021-04-01 00:24:55,2021-04-01 00:34:33,9.0,1.96,4.591836929321289
2,2021-04-01 00:19:16,2021-04-01 00:21:46,4.5,0.77,5.844155788421631
2,2021-04-01 00:25:11,2021-04-01 00:31:53,11.5,3.65,3.1506848335266113
1,2021-04-01 00:27:53,2021-04-01 00:47:03,26.5,8.9,2.9775280952453613
2,2021-04-01 00:24:24,2021-04-01 00:37:50,12.0,2.98,4.026845455169678
1,2021-04-01 00:19:18,2021-04-01 00:41:25,28.0,8.9,3.1460673809051514


## Expressive SQL for Row Level Changes
With Iceberg tables, `DELETE` queries can be used to perform row-level deletes. This is as simple as providing the table name and a `WHERE` predicate. If the filter matches an entire partition of the table, Iceberg will intelligently perform a metadata-only operation where it simply deletes the metadata for that partition.

Let's perform a row-level delete for all rows that have a `fare_per_distance_unit` greater than 4 or a `distance` greater than 2. This should leave us with relatively short trips that have a relatively high fare per distance traveled.

In [ ]:
%%sql

DELETE FROM nyc.taxis
WHERE fare_per_distance_unit > 4.0 OR distance > 2.0

There are some fares that have a `null` for `fare_per_distance_unit` due to the distance being `0`. Let's remove those as well.

In [ ]:
%%sql

DELETE FROM nyc.taxis
WHERE fare_per_distance_unit is null

In [ ]:
%%sql

SELECT
VendorID
,tpep_pickup_datetime
,tpep_dropoff_datetime
,fare
,distance
,fare_per_distance_unit
FROM nyc.taxis

In [ ]:
%%sql

SELECT COUNT(*) as cnt
FROM nyc.taxis

## Partitioning

A table’s partitioning can be updated in place and applied only to newly written data. Query plans are then split, using the old partition scheme for data written before the partition scheme was changed, and using the new partition scheme for data written after. People querying the table don’t even have to be aware of this split. Simple predicates in WHERE clauses are automatically converted to partition filters that prune out files with no matches. This is what’s referred to in Iceberg as *Hidden Partitioning*.

In [ ]:
%%sql

ALTER TABLE nyc.taxis
ADD PARTITION FIELD VendorID

## Metadata Tables

Iceberg tables contain very rich metadata that can be easily queried. For example, you can retrieve the manifest list for any snapshot, simply by querying the table's `snapshots` table.

In [7]:
%%sql

SELECT snapshot_id, manifest_list
FROM nyc.taxis.snapshots

Py4JJavaError: An error occurred while calling o94.collectToPython.
: org.apache.iceberg.exceptions.NotFoundException: File does not exist: file:/tmp/polaris/nyc/taxis/metadata/00000-05496509-a6b0-4608-82c6-7ac6216c4a5a.metadata.json
	at org.apache.iceberg.hadoop.HadoopInputFile.lazyStat(HadoopInputFile.java:164)
	at org.apache.iceberg.hadoop.HadoopInputFile.getStat(HadoopInputFile.java:200)
	at org.apache.iceberg.DataFiles$Builder.withInputFile(DataFiles.java:215)
	at org.apache.iceberg.StaticDataTask.<init>(StaticDataTask.java:60)
	at org.apache.iceberg.StaticDataTask.of(StaticDataTask.java:46)
	at org.apache.iceberg.SnapshotsTable.task(SnapshotsTable.java:61)
	at org.apache.iceberg.SnapshotsTable.access$100(SnapshotsTable.java:29)
	at org.apache.iceberg.SnapshotsTable$SnapshotsTableScan.planFiles(SnapshotsTable.java:93)
	at org.apache.iceberg.BatchScanAdapter.planFiles(BatchScanAdapter.java:124)
	at org.apache.iceberg.spark.source.SparkPartitioningAwareScan.tasks(SparkPartitioningAw

The `files` table contains loads of information on data files, including column level statistics such as null counts, lower bounds, and upper bounds.

In [ ]:
%%sql

SELECT file_path, file_format, record_count, null_value_counts, lower_bounds, upper_bounds
FROM nyc.taxis.files

## Time Travel

The history table lists all snapshots and which parent snapshot they derive from. The `is_current_ancestor` flag let's you know if a snapshot is part of the linear history of the current snapshot of the table.

In [ ]:
%%sql

SELECT *
FROM nyc.taxis.history

You can time-travel by altering the `current-snapshot-id` property of the table to reference any snapshot in the table's history. Let's revert the table to it's original state by traveling to the very first snapshot ID.

In [ ]:
%%sql --var df

SELECT *
FROM nyc.taxis.history

In [ ]:
original_snapshot = df.head().snapshot_id
spark.sql(f"CALL system.rollback_to_snapshot('nyc.taxis', {original_snapshot})")
original_snapshot

In [ ]:
%%sql

SELECT
VendorID
,tpep_pickup_datetime
,tpep_dropoff_datetime
,fare
,distance
,fare_per_distance_unit
FROM nyc.taxis

Another look at the history table shows that the original state of the table has been added as a new entry
with the original snapshot ID.

In [ ]:
%%sql

SELECT *
FROM nyc.taxis.history

In [ ]:
%%sql

SELECT COUNT(*) as cnt
FROM nyc.taxis